<a href="https://colab.research.google.com/github/lschreiber9/Training-BERT/blob/main/Training_the_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training the model

Datasets

In [ ]:
import pandas as pd

datasets = {
    "climate_specificity": {
        "train": "hf://datasets/climatebert/climate_specificity/data/train-00000-of-00001-298fad749f8929f7.parquet",
        "test": "hf://datasets/climatebert/climate_specificity/data/test-00000-of-00001-2588e03729a1bfe7.parquet"
    },
    "climate_sentiment": {
        "train": "hf://datasets/climatebert/climate_sentiment/data/train-00000-of-00001-04b49ae22f595095.parquet",
        "test": "hf://datasets/climatebert/climate_sentiment/data/test-00000-of-00001-3f9f7af4f5914b8e.parquet"
    },
    "climate_commitments_actions": {
        "train": "hf://datasets/climatebert/climate_commitments_actions/data/train-00000-of-00001-2044cce9e261c6b3.parquet",
        "test": "hf://datasets/climatebert/climate_commitments_actions/data/test-00000-of-00001-77f76c0960abb9c6.parquet"
    },
    "environmental_claims": {
        "train": "hf://datasets/climatebert/environmental_claims/data/train-00000-of-00001-98aa5228a06a17d0.parquet",
        "validation": "hf://datasets/climatebert/environmental_claims/data/validation-00000-of-00001-2553e47d408fab28.parquet",
        "test": "hf://datasets/climatebert/environmental_claims/data/test-00000-of-00001-79fd931297fff765.parquet"
    },
    "climate_detection": {
        "train": "hf://datasets/climatebert/climate_detection/data/train-00000-of-00001-4b831beb8839bf3e.parquet",
        "test": "hf://datasets/climatebert/climate_detection/data/test-00000-of-00001-87f8706e009e9b75.parquet"
    }
}

# Function to load data for a specific dataset
def load_data(dataset_name):
    if dataset_name not in datasets:
        raise ValueError(f"Dataset '{dataset_name}' is not defined.")

    dataset_info = datasets[dataset_name]
    train_df = pd.read_parquet(dataset_info["train"])
    test_df = pd.read_parquet(dataset_info["test"])

    return train_df, test_df

# Load data for each dataset and assign variables dynamically
for dataset_name in datasets.keys():
    globals()[f"train_{dataset_name}"], globals()[f"test_{dataset_name}"] = load_data(dataset_name)

# Check the results
print("Specificity Train Data:")
print(train_climate_specificity.head())

print("\nSentiment Test Data:")
print(test_climate_sentiment.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Specificity Train Data:
                                                text  label
0  − Scope 3: Optional scope that includes indire...      1
1  The Group is not aware of any noise pollution ...      0
2  Global climate change could exacerbate certain...      0
3  Setting an investment horizon is part and parc...      0
4  Climate change the physical impacts of climate...      0

Sentiment Test Data:
                                                text  label
0  Sustainable strategy ‘red lines’ For our susta...      0
1  Verizon’s environmental, health and safety man...      1
2  In 2019, the Company closed a series of transa...      1
3  In December 2020, the AUC approved the Electri...      0
4  Finally, there is a reputational risk linked t...      0


Import Tokenizer

In [ ]:
from transformers import BertTokenizer

# Load the tokenizer for bert-base-uncased
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


Preprocess Data so fit for BERT

In [ ]:
def preprocess_data(df, text_column, label_column, max_length=512):
    # Tokenize the text column
    encodings = tokenizer(
        df[text_column].tolist(),  # Texts to tokenize
        truncation=True,          # Truncate text to max_length
        padding=True,             # Pad shorter sequences
        max_length=max_length,    # Maximum sequence length
        return_tensors="pt"       # Return PyTorch tensors
    )

    # Extract labels
    labels = df[label_column].tolist()

    return encodings, labels

Preprocess climate specificity dataset

In [ ]:
preprocessed_data = {}

for dataset_name in datasets.keys():
    train_df = globals()[f"train_{dataset_name}"]
    test_df = globals()[f"test_{dataset_name}"]

    train_encodings, train_labels = preprocess_data(train_df, text_column="text", label_column="label")
    test_encodings, test_labels = preprocess_data(test_df, text_column="text", label_column="label")

    # Save the preprocessed data in a dictionary
    preprocessed_data[dataset_name] = {
        "train": (train_encodings, train_labels),
        "test": (test_encodings, test_labels)
    }


PyTorch necessary for training


In [ ]:
import torch
from torch.utils.data import Dataset

class ClimateDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Return encodings and labels for a single data point
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item


Tokenized Data and labels into ClimateDataset


In [ ]:
from torch.utils.data import DataLoader

dataloaders = {}

for dataset_name, splits in preprocessed_data.items():
    train_encodings, train_labels = splits["train"]
    test_encodings, test_labels = splits["test"]

    train_dataset = ClimateDataset(train_encodings, train_labels)
    test_dataset = ClimateDataset(test_encodings, test_labels)

    dataloaders[dataset_name] = {
        "train": DataLoader(train_dataset, batch_size=16, shuffle=True),
        "test": DataLoader(test_dataset, batch_size=64, shuffle=False)
    }


Choosing the model


In [ ]:
from transformers import BertForSequenceClassification

# Load BERT model for binary classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define and optimize loss function

In [ ]:
from transformers import AdamW
from torch.optim import AdamW

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

CPU or GPU?

In [ ]:
# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Training Loop for one Dataset

In [ ]:
def train_model(model, optimizer, train_loader, test_loader, device, epochs=3):
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        total_loss = 0
        correct = 0

        for batch in train_loader:
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            predictions = torch.argmax(logits, dim=1)
            correct += (predictions == labels).sum().item()

        # Calculate training accuracy
        accuracy = correct / len(train_loader.dataset)
        print(f"Epoch {epoch + 1}: Loss = {total_loss:.4f}, Accuracy = {accuracy:.4f}")

    # Evaluate the model after training
    evaluate_model(model, test_loader, device)


Evaluation function

In [ ]:
def evaluate_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total_loss = 0

    with torch.no_grad():
        for batch in test_loader:
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()
            predictions = torch.argmax(logits, dim=1)
            correct += (predictions == labels).sum().item()

    accuracy = correct / len(test_loader.dataset)
    print(f"Test Loss = {total_loss:.4f}, Test Accuracy = {accuracy:.4f}")


Training for each Dataset

In [ ]:
import gc
from transformers import BertForSequenceClassification, AdamW
from datetime import datetime

for dataset_name, loaders in dataloaders.items():
    print(f"Training on {dataset_name} dataset...")

    # Load dataloaders
    train_loader = loaders["train"]
    test_loader = loaders["test"]

    # Dynamically set `num_labels` based on the dataset
    num_labels = 3 if dataset_name == "climate_sentiment" else 2

    # Reload the model with the correct number of labels
    try:
        model = BertForSequenceClassification.from_pretrained(
            "bert-base-uncased", num_labels=num_labels
        )
        optimizer = AdamW(model.parameters(), lr=5e-5)
        model.to(device)

        # Train the model
        print(f"Started training on {dataset_name} at {datetime.now()}")
        train_model(model, optimizer, train_loader, test_loader, device, epochs=3)

        # Save the model for the specific dataset
        save_path = f"/content/drive/MyDrive/Big Data Project/NLP analysis/{dataset_name}_model"
        model.save_pretrained(save_path)
        print(f"Model for {dataset_name} saved at {save_path}!")

    except RuntimeError as e:
        print(f"Error during training on {dataset_name}: {e}")
        torch.cuda.empty_cache()  # Clear memory and continue
        continue

    # Free memory after training on the current dataset
    del train_loader, test_loader
    del model, optimizer
    gc.collect()
    torch.cuda.empty_cache()

print("Training completed for all datasets.")


# Model implementation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import AutoTokenizer

base_model = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model_dir = "/content/drive/MyDrive/Big Data Project/NLP analysis/climate_sentiment_model"
tokenizer.save_pretrained(model_dir)


Format to JSON

In [ ]:
import os
import json

folder_path = "/content/drive/MyDrive/Big Data Project/NLP analysis/Extracted Text"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    if os.path.isfile(file_path) and filename.endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()

            json_filename = filename.replace(".txt", ".json")
            json_file_path = os.path.join(folder_path, json_filename)

            with open(json_file_path, "w", encoding="utf-8") as json_file:
                json.dump({"content": content}, json_file, ensure_ascii=False, indent=4)

print("Alle TXT-Dateien wurden erfolgreich in JSON-Dateien umgewandelt.")

Alle TXT-Dateien wurden erfolgreich in JSON-Dateien umgewandelt.


Sentiment score

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Nutze Gerät: {device}")

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Pfade zu den extrahierten Texten und dem Modell
text_folder = '/content/drive/MyDrive/Big Data Project/NLP analysis/Extracted Text'
model_dir = "/content/drive/MyDrive/Big Data Project/NLP analysis/climate_sentiment_model"

# Modell und Tokenizer laden
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(device)


Nutze Gerät: cuda


Label mapping

In [ ]:
config_path = os.path.join(model_dir, "config.json")
with open(config_path, "r") as f:
    config = json.load(f)
label_mapping = config.get("id2label", {})

print(f"Geladene Labels: {label_mapping}")


Geladene Labels: {'0': 'LABEL_0', '1': 'LABEL_1', '2': 'LABEL_2'}


Analysis and Prediction

In [ ]:
def predict_label(segment):
    """Label und Wahrscheinlichkeiten für ein Segment vorhersagen"""
    inputs = tokenizer(segment, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1).detach().cpu().numpy()[0]

    # Vorhersageklasse und Wahrscheinlichkeiten
    predicted_class = int(torch.argmax(logits, dim=1).item())

    # Mapping der Labels: 0 -> 0 (neutral), 1 -> -1 (risk), 2 -> 1 (opportunity)
    label_mapping = {0: 0, 1: -1, 2: 1}
    mapped_label = label_mapping[predicted_class]

    return mapped_label, probabilities

def analyze_file(file_path):
    """JSON-Datei analysieren, Absätze auswerten und einen Gesamtindex berechnen"""
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
        text = data.get("content", "")

    # Text in Paragraphen splitten
    segments = split_text_into_segments(text)

    segment_results = []
    for segment in segments:
        label, probabilities = predict_label(segment)
        segment_results.append({"segment": segment, "label": label, "probabilities": probabilities})

    # Gesamtindex berechnen: Mittelwert der gemappten Labels
    overall_index = sum(r["label"] for r in segment_results) / len(segment_results)
    return overall_index, segment_results

# Verzeichnis mit JSON-Dateien
text_folder = "/content/drive/MyDrive/Big Data Project/NLP analysis/Extracted Text"
results = {}

# Alle Dateien im Verzeichnis analysieren
for filename in os.listdir(text_folder):
    file_path = os.path.join(text_folder, filename)
    if os.path.isfile(file_path) and filename.endswith(".json"):
        overall_index, segment_results = analyze_file(file_path)
        results[filename] = {"overall_index": overall_index, "segment_results": segment_results}

# Ergebnisse ausgeben
for file, result in results.items():
    print(f"{file}: Overall Index = {result['overall_index']:.2f}")
    for i, segment_result in enumerate(result["segment_results"]):
        print(f"  Segment {i + 1}: Label = {segment_result['label']}, Probabilities = {segment_result['probabilities']}")

Save results

In [ ]:
import csv
import os
import re

csv_file_path = "/content/drive/MyDrive/Big Data Project/NLP analysis/sentiment_results.csv"

sorted_results = sorted(
    filtered_results.items(),
    key=lambda x: tuple(map(int, re.match(r'^(\d+)\.(\d+)', x[0]).groups())) if re.match(r'^(\d+)\.(\d+)', x[0]) else (float('inf'), float('inf'))
)

# Verzeichnis erstellen, falls nicht vorhanden
os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)

# Ergebnisse in der CSV-Datei speichern
with open(csv_file_path, mode='w', encoding='utf-8', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Filename", "Overall Index"])  # Kopfzeile schreiben

    # Sortierte Ergebnisse in die CSV schreiben, Score multiplizieren
    for file, result in sorted_results:
        final_score = result['overall_index'] * 5
        writer.writerow([file, final_score])

Commitment Score